In [4]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.stats as st
import seaborn as sns
import pandas as pd

In [76]:
# Эксперимент
rs = np.random.RandomState(42)

alpha = 0.05
m = 10000
m0 = 8500
n = 20

X = rs.randn(m,n)
X[m0:]+=1

### Без поправок

In [77]:
# zero mean
rejected = 0
V = 0
R = 0
for x in X[:m0]:
    if (st.ttest_1samp(x, 0)[1] <= alpha):
        rejected += 1
R += rejected
V = rejected
print ('True: H0; H1 non-accepted:', m0 - rejected)
print ('True: H0; H0 rejected:', rejected)

# non-zero mean
rejected = 0
for x in X[m0:]:
    if (st.ttest_1samp(x, 0)[1] <= alpha):
        rejected += 1
R+= rejected
print ('True: H1; H1 non-accepted:', m-m0 - rejected)
print ('True: H1; H0 rejected:', rejected)

True: H0; H1 non-accepted: 8053
True: H0; H0 rejected: 447
True: H1; H1 non-accepted: 17
True: H1; H0 rejected: 1483


In [78]:
# наблюдаем за FDR
FDR = V/R
print("FDR = ", FDR)

FDR =  0.23160621761658032


In [ ]:
#Без поправок при данной alpha работает не очень, если уменьшить alpha, будет лучше результат

### Холм

In [79]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])     
sorted_p = np.sort(np.array(p_values, dtype = float))

V = 0
R = 0
# считаю R
for i in range(m):
    if (sorted_p[i] >= alpha/(m-i)):
        where_stop = sorted_p[i]
        break
    else:
        R += 1
# Считаю V
for x in X[:m0]:
    if st.ttest_1samp(x, 0)[1] < where_stop:
        V += 1
FDR = V/R

print("V = ", V, "R = ", R, "FDR = ", FDR)

V =  0 R =  187 FDR =  0.0


In [ ]:
# Получается маленькая FDR, но это потому что очень редко отвергаем гипотезу, только в очень маленьких p_value
# Работает лучше при увеличении исходного alpha
# Следующая поправка работает лучше

###  Бенджамини-Хохберг

In [80]:
p_values = []
for x in X:
     p_values.append(st.ttest_1samp(x, 0)[1])     
sorted_p = np.sort(np.array(p_values, dtype = float))

V = 0
R = 0

# считаю R
for i in range(m):
    if (sorted_p[i] >= alpha*(i+1)/m):
        where_stop = sorted_p[i]
        break
    else:
        R += 1
# Считаю V
for x in X[:m0]:
    if st.ttest_1samp(x, 0)[1] < where_stop:
        V += 1
FDR = V/R

print("V = ", V, "R = ", R, "FDR = ", FDR)

V =  67 R =  1435 FDR =  0.046689895470383276


In [ ]:
# Здесь намного лучше, отвергаем почти все которые нужно отвергнуть, и совсем немного отвергаем правильных 
#нулевых гипотез